In [ ]:
import sys
import os

# TODO: this is a hack, change this for editable install
module_path = os.path.abspath(os.path.join('../..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from src.functions import *

from src.loss import cross_entropy_loss

In [5]:
def test_n_samples_function(model_name: str, n_samples, max_len: int = 10, n_epochs: int = 50000, batch_size: int = 256, learning_rate: float = 1e-04, output_dir: str = None):
    model = generateModel(model_name, max_len)
    model.setRandomWeights()
    X, Y = generateAndEncodeData(model_name, max_len, n_samples, removeDuplicates=True)

    X_train = X[:int(0.85*len(X))]
    Y_train = Y[:int(0.85*len(Y))]
    X_val = X[int(0.85*len(X)):]
    Y_val = Y[int(0.85*len(Y)):]

    return model.train(X_train,Y_train, n_epochs, batch_size, learning_rate, False, X_val, Y_val, 0, 10, loss_fn=cross_entropy_loss, output_dir=output_dir)

In [ ]:
from concurrent.futures import ThreadPoolExecutor, as_completed

def run_test(model_name, n_samples):
    print(f"Testing {model_name} with {n_samples} samples...")
    metrics, validation = test_n_samples_function(model_name, n_samples, output_dir=f"saved_data/{model_name}/{n_samples}/")
    return model_name, n_samples, metrics, validation

max_workers = 3
with ThreadPoolExecutor(max_workers=max_workers) as executor:
    futures = []
    for model_name in getAcceptedNamesAndInput().keys():
        print(f"Testing {model_name}...")
        for n_samples in [5000, 10000, 25000, 50000, 100000]:
            future = executor.submit(run_test, model_name, n_samples)
            futures.append(future)
            break
        break

    for future in as_completed(futures):
        model_name, n_samples, metrics, validation = future.result()
        print(f"Completed {model_name} with {n_samples} samples")

Testing reverse...
Testing reverse with 5000 samples...
